# Initialisation

In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
import numpy as np
#import scipy
from scipy.special import zeta
from time import time
from astropy import cosmology as cosmo
from astropy import units as u
from astropy import constants as c
from astropy.visualization import quantity_support
quantity_support()

## Models

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from models import flcdm, coasting

In [ ]:
coasting.H0, flcdm.H0

## Data

In [ ]:
SN_z, SN_err_z, SN_mu, SN_err_mu = np.loadtxt('data/Pantheon+_Data.dat', usecols=(2, 3, 10, 11), unpack=True, skiprows=1)
GRB1 = np.loadtxt('data/GRB_RNN.dat', skiprows=1)
GRB2 = np.loadtxt('data/GRB_RNN_2.dat', skiprows=1)
GRB_z = np.concatenate([GRB1[:, 0], GRB1[:, 3], GRB1[:, 6], GRB2[:, 0]])
GRB_mu = np.concatenate([GRB1[:, 1], GRB1[:, 4], GRB1[:, 7], GRB2[:, 1]])
GRB_err_mu = np.concatenate([GRB1[:, 2], GRB1[:, 5], GRB1[:, 8], GRB2[:, 2]])
QSO_z, QSO_mu, QSO_err_mu = np.loadtxt('data/table3QSO.dat', usecols=(3, 11, 12), unpack=True, skiprows=1)
#QSO_err_mu *= 2

def Dl_Mpc(mu):
    return 1e-5 * 10**(.2*mu)

#SN_Dl = Dl_Mpc(SN_mu)
#GRB_Dl = Dl_Mpc(GRB_mu)
#QSO_Dl = Dl_Mpc(QSO_mu)


all_z = np.concatenate([SN_z, QSO_z, GRB_z])
all_mu = np.concatenate([SN_mu, QSO_mu, GRB_mu])
all_err_mu = np.concatenate([SN_err_mu, QSO_err_mu, GRB_err_mu])

#all_z = SN_z
#all_mu = SN_mu
#all_err_mu = SN_err_mu
#all_z = QSO_z
#all_mu = QSO_mu
#all_err_mu = QSO_err_mu
#all_z = GRB_z
#all_mu = GRB_mu
#all_err_mu = GRB_err_mu


In [ ]:
thetaBAO = np.array([
    0.11,19.8,3.26,
    0.235,9.06,0.23,
    0.365,6.33,0.22,
    0.45,4.77,0.17,
    0.47,5.02,0.25,
    0.49,4.99,0.21,
    0.51,4.81,0.17,
    0.53,4.29,0.30,
    0.55,4.25,0.25,
    0.57,4.59,0.36,
    0.59,4.39,0.33,
    0.61,3.85,0.31,
    0.63,3.90,0.43,
    0.65,3.55,0.16,
    2.225,1.77,0.31])

theta_z = thetaBAO[::3]
theta_z_err = np.array([0.005,0.035,0.025,0.01,0.005,0.01,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.01,0.025])
theta_angle_deg = thetaBAO[1::3]
theta_angle_err = thetaBAO[2::3]
#theta_relative_err = theta_angle_err / theta_angle_deg
#theta_DM = 180/np.pi/theta_angle_deg

# Plot cosmic evolution

## Distances

In [ ]:
fig_name = 'distances'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(8, 9))
axes = fig.subplots(nrows=3, ncols=1, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )

ax = axes[0, 0]
ax.set_ylabel(r'radial comoving distance $D_C$ [Mpc]')
ax.set_yscale('log')
ax.set_ylim(3e-1, 3e5)

ax.plot(coasting.z_t, coasting.Dc_t.to_value(u.Mpc), 'r-', label='coasting')
ax.plot(flcdm.z_t, flcdm.Dc_t.to_value(u.Mpc), 'r--', label=r'flat $\Lambda$CDM')
ax.legend()

#ax.plot(flcdm.z_t, cosmo.Planck18.comoving_distance(flcdm.z_t).to_value(u.Mpc), 'k+', alpha=.1)
#DH = (c.c/coasting.H0).to_value(u.Mpc)
#ax.plot(coasting.z_t, np.log(1+coasting.z_t)*DH, 'k+', alpha=.1)


ax = axes[1, 0]
ax.set_ylabel(r'angular diameter distance $D_A$ [Mpc]')
ax.set_yscale('log')
ax.set_ylim(3e-1, 3e3)

ax.plot(coasting.z_t, coasting.Dm_t.to_value(u.Mpc) / (1+coasting.z_t), 'k-', label='coasting')
ax.plot(flcdm.z_t, flcdm.Dm_t.to_value(u.Mpc) / (1+flcdm.z_t), 'k--', label=r'flat $\Lambda$CDM')
ax.legend()

#ax.plot(flcdm.z_t, cosmo.Planck18.angular_diameter_distance(flcdm.z_t).to_value(u.Mpc), 'k+', alpha=.1)
#z_cmb = np.interp(flcdm.t_cmb, flcdm.t, flcdm.z_t)
#print(z_cmb, cosmo.Planck18.angular_diameter_distance(z_cmb).to_value(u.Mpc)*(1+z_cmb))
#Ok = .05
#DH = (c.c/coasting.H0).to_value(u.Mpc)
#ax.plot(coasting.z_t, np.abs(np.sin(np.log((1+coasting.z_t)) * np.sqrt(Ok)) / np.sqrt(Ok))*DH / (1+coasting.z_t), 'k+', alpha=.1)
#z_cmb = 1/(coasting.t_cmb * coasting.H0).to_value(u.dimensionless_unscaled)
#print(z_cmb, np.abs(np.sin(np.log((1+z_cmb)) * np.sqrt(Ok)) / np.sqrt(Ok))*DH)

ax = axes[2, 0]
ax.set_ylabel(r'ratio (coasting / flat $\Lambda$CDM)')
#ax.set_yscale('log')
ax.set_ylim(.75, 1.25)

z = np.logspace(-4, 4, 401)
ax.plot(z, coasting.comoving_distance(z) / flcdm.comoving_distance(z), 'r-', label=r'$D_C$')
ax.plot(z, coasting.angular_diameter_distance(z) / flcdm.angular_diameter_distance(z), 'k-', label='$D_A$ and $D_L$')
ax.legend()


ax.set_xlabel('redshift $z$')
ax.set_xscale('log')
ax.set_xlim(3e-4, 3e3)

for ax in axes.flatten():
    ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
    ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
    ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
    ax.grid(True, which='both')

fig.set_tight_layout(True)
plt.show()
plt.savefig('figures/distances.pdf')

## Densities

In [ ]:
fig_name = 'evolution'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(8, 8))
axes = fig.subplots(nrows=2, ncols=1, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )
alpha = 1

ax = axes[0, 0]
ax.set_ylabel(r'density $\rho$ [M$_\odot$ Mpc$^{-3}$]')

def mass_to_E(m):
    return((m*c.c**2 * u.Msun/u.Mpc**3).to_value(u.GeV/u.cm**3))
def E_to_mass(E):
    return((E/c.c**2 * u.GeV/u.cm**3).to_value(u.Msun/u.Mpc**3))
secax = ax.secondary_yaxis('right', functions=(mass_to_E, E_to_mass))
secax.set_ylabel(r'energy density $\rho c^2$ [GeV cm$^{-3}$]')


ax.set_yscale('log')
ax.set_xscale('log')

legends = []
for model, title, ls, bbox in zip([flcdm, coasting],
                                  [r'Flat $\Lambda$CDM', 'Coasting'],
                                  ['--', '-'],
                                  [(0.45, 0.5, 0.5, 0.5), (0.72, 0.5, 0.5, 0.5)],
                                 ):
    t = model.t.to_value(u.yr)
    rho_c = model.rho_crit_t.to_value(u.Msun/u.Mpc**3)
    rho_de = np.exp(model.ln_Ode_t) * rho_c
    rho_m = np.exp(model.ln_Om_t) * rho_c
    rho_b = rho_m * model.Ob0/model.Om0
    rho_gamma = np.exp(model.ln_Ogamma_t) *rho_c
    
    l1, = ax.plot(t, rho_de, c='r', ls=ls, alpha=alpha, label='dark energy')
    l2, = ax.plot(t, rho_m - rho_b, c='k', ls=ls, alpha=alpha, label='dark matter')
    l3, = ax.plot(t, rho_b, c='c', ls=ls, alpha=alpha, label='baryons (atoms)')
    l4, = ax.plot(t, rho_gamma, c='y', ls=ls, alpha=alpha, label='photons')
    legends.append(ax.legend(handles=[l1, l2, l3, l4], title=title, loc='upper left', bbox_to_anchor=bbox))

ax.add_artist(legends[0])
ax.set_xlim(3e3, 3e10)
ax.set_ylim(3e6, 3e25)
ax.set_yticks(np.logspace(7, 25, 19), minor=True, labels='')

ax = axes[1, 0]
ax.set_ylabel('ionisation fraction $x$')

ax.plot(flcdm.t.to_value(u.yr), flcdm.x_ion_t, 'c--', alpha=alpha)
ax.plot(coasting.t.to_value(u.yr), coasting.x_ion_t, 'c-', alpha=alpha)

ax.plot(flcdm.t_cmb.to_value(u.yr), np.interp(flcdm.t_cmb, flcdm.t, flcdm.x_ion_t), 'cd', fillstyle='none',
           label=r'Flat $\Lambda$CDM: $t_{\rm CMB} \sim$'+f'{flcdm.t_cmb.to_value(u.Myr):.3f} Myr')
ax.plot(coasting.t_cmb.to_value(u.yr), np.interp(coasting.t_cmb, coasting.t, coasting.x_ion_t), 'cd',
        label=r'Coasting: $t_{\rm CMB} \sim$'+f'{coasting.t_cmb.to_value(u.Myr):.3f} Myr')

l = ax.legend().get_lines()
l[0].set_ls('--')
l[1].set_ls('-')
#h.set_visible(False)

ax.set_xscale('log')
ax.set_xlabel('cosmic time $t$ [yr]')


for ax in axes.flatten():
    #ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
    ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
    ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
    ax.grid(True, which='both')
    #ax.axvline(t_cmb_p18, c='r', ls='--')
    #ax.axvline(t_cmb_model, c='k', ls='-')
fig.set_tight_layout(True)
plt.show()
plt.savefig('figures/evolution.pdf')

# Compare with observations

In [ ]:
from models import flcdm, coasting

def plot_DL_DH_obs(ax, z, mu, err_mu, **kwargs):
    #scale = 3e5 * z
    scale = z
    y = Dl_Mpc(mu) / scale
    dy_low = y-Dl_Mpc(mu-err_mu) / scale
    dy_high = Dl_Mpc(mu+err_mu) / scale - y
    ax.errorbar(z, y, yerr=(dy_low, dy_high), fmt='.', **kwargs)
    
    
fig_name = 'observations'

plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(8, 9))
axes = fig.subplots(nrows=2, ncols=1, squeeze=False,
                    sharex=True, sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )

ax = axes[0, 0]
ax.set_ylabel(r'$D_L(z)/z$ [Mpc]')
ax.set_ylim(0, 2.1e4)
#ax.set_yscale('log')

plot_DL_DH_obs(ax, SN_z, SN_mu, SN_err_mu, c='c', alpha=.05, label='SNe')
plot_DL_DH_obs(ax, QSO_z, QSO_mu, QSO_err_mu, c='y', alpha=.1, label='QSO')
plot_DL_DH_obs(ax, GRB_z, GRB_mu, GRB_err_mu, c='r', alpha=.2, label='GRB')

flcdm.fit_mu(SN_z, SN_mu, SN_err_mu)
flcdm.fit_mu(QSO_z, QSO_mu, QSO_err_mu)
flcdm.fit_mu(GRB_z, GRB_mu, GRB_err_mu)
flcdm.fit_mu(all_z, all_mu, all_err_mu)
print()
coasting.fit_mu(SN_z, SN_mu, SN_err_mu)
coasting.fit_mu(QSO_z, QSO_mu, QSO_err_mu)
coasting.fit_mu(GRB_z, GRB_mu, GRB_err_mu)
coasting.fit_mu(all_z, all_mu, all_err_mu)
print()
ax.plot(flcdm.z_t, flcdm.Dm_t.to_value(u.Mpc) * (1+flcdm.z_t)/flcdm.z_t, 'k--', label=r'flat $\Lambda$CDM')
ax.plot(coasting.z_t, coasting.Dm_t.to_value(u.Mpc) * (1+coasting.z_t)/coasting.z_t, 'k-', label='coasting')

ax.legend()


ax = axes[1, 0]
ax.set_ylabel(r'$\theta_{BAO}$ [$^\circ$]')
ax.set_ylim(0, 24)
#ax.set_yscale('log')

ax.errorbar(theta_z, theta_angle_deg, theta_angle_err, theta_z_err, fmt='k.', label='Nunes et al. (2020)')

ax.plot(flcdm.z_t, (flcdm.r_d / flcdm.Dm_t * (1+flcdm.z_t) * u.rad).to_value(u.deg), 'k--', label=r'flat $\Lambda$CDM')
ax.plot(coasting.z_t, (coasting.r_d / coasting.Dm_t * (1+coasting.z_t) * u.rad).to_value(u.deg), 'k-', label=r'coasting')

flcdm.fit_theta_BAO(theta_z, theta_angle_deg, theta_angle_err)
coasting.fit_theta_BAO(theta_z, theta_angle_deg, theta_angle_err)

ax.legend()


ax.set_xlabel('redshift $z$')
ax.set_xscale('log')
ax.set_xlim(1e-3, 20)
#ax.set_xlim(-.2, 8.2)

for ax in axes.flatten():
    ax.tick_params(which='both', direction='in')
    ax.grid(alpha=.5)
fig.set_tight_layout(True)
plt.show()
#plt.savefig('DL.pdf')


In [ ]:
u.deg/u.rad